In [71]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time

In [72]:
df=pd.read_csv("data.csv")

In [73]:
df2=df.pop('is_goal')

In [74]:
df=df.fillna(-1)

# converting season to int
ses=df['game_season'].tolist()
for i in range(len(ses)):
    if type(ses[i])==type('abc'):
        ses[i]=int(ses[i].split('-')[0])
s = pd.Series(ses)
df1 = pd.DataFrame({'game_season':(s)})
df.pop('game_season')
df=df.join(df1)

In [75]:
# giving labels to area of shot
ids=df['area_of_shot'].tolist()
s = pd.Series(ids)
labels, levels = pd.factorize(s)
df1 = pd.DataFrame({'area_of_shot':(labels)})
df.pop('area_of_shot')
df=df.join(df1)

In [76]:
# giving labels to shot basics
ids=df['shot_basics'].tolist()
s = pd.Series(ids)
labels, levels = pd.factorize(s)
df1 = pd.DataFrame({'shot_basics':(labels)})
df.pop('shot_basics')
df=df.join(df1)

In [77]:
# giving labels to range of shots
ids=df['range_of_shot'].tolist()
s = pd.Series(ids)
labels, levels = pd.factorize(s)
df1 = pd.DataFrame({'range_of_shot':(labels)})
df.pop('range_of_shot')
df=df.join(df1)

In [78]:
# giving labels to team name
ids=df['team_name'].tolist()
s = pd.Series(ids)
labels, levels = pd.factorize(s)
df1 = pd.DataFrame({'team_name':(labels)})
df.pop('team_name')
df=df.join(df1)

In [79]:
# converting date to int
ses=df['date_of_game'].tolist()
for i in range(len(ses)):
    if type(ses[i])==type('abc'):
        ses[i]=''.join((ses[i].split('-')))
        ses[i]=int(ses[i])
s = pd.Series(ses)
df1 = pd.DataFrame({'date_of_game':(s)})
df.pop('date_of_game')
df=df.join(df1)

In [80]:
# giving labels to home/away
ses=df['home/away'].tolist()
for i in range(len(ses)):
    if type(ses[i])==type('abc'):
        x=ses[i].split(' ')[1]
        if(x=='@'):
            ses[i]=0
        elif(x=='vs.'):
            ses[i]=1
s = pd.Series(ses)
df1 = pd.DataFrame({'home/away':(s)})
df.pop('home/away')
df=df.join(df1)

In [81]:
# giving labels to type of shot
ids=df['type_of_shot'].tolist()
s = pd.Series(ids)
labels, levels = pd.factorize(s)
df1 = pd.DataFrame({'type_of_shot':(labels)})
df.pop('type_of_shot')
df=df.join(df1)

In [82]:
# giving labels to combined shot
ids=df['type_of_combined_shot'].tolist()
s = pd.Series(ids)
labels, levels = pd.factorize(s)
df1 = pd.DataFrame({'type_of_combined_shot':(labels)})
df.pop('type_of_combined_shot')
df=df.join(df1)

In [83]:
# converting latitude longitude to int values
ses=df['lat/lng'].tolist()
for i in range(len(ses)):
    if type(ses[i])==type('abc'):
        ses[i]=float(ses[i].split(',')[0])+float(ses[i].split(',')[1])
s = pd.Series(ses)
df1 = pd.DataFrame({'lat/lng':(s)})
df.pop('lat/lng')
df=df.join(df1)

In [84]:
stats = df.describe()
stats=stats.transpose()

In [85]:
def norm(x):
    return (x - stats['mean']) / stats['std']
normed_df = norm(df)
df=df.join(df2)

In [86]:
df.pop('team_id')

0        1610612747
1        1610612747
2        1610612747
3        1610612747
4        1610612747
5        1610612747
6        1610612747
7        1610612747
8        1610612747
9        1610612747
10       1610612747
11       1610612747
12       1610612747
13       1610612747
14       1610612747
15       1610612747
16       1610612747
17       1610612747
18       1610612747
19       1610612747
20       1610612747
21       1610612747
22       1610612747
23       1610612747
24       1610612747
25       1610612747
26       1610612747
27       1610612747
28       1610612747
29       1610612747
            ...    
30667    1610612747
30668    1610612747
30669    1610612747
30670    1610612747
30671    1610612747
30672    1610612747
30673    1610612747
30674    1610612747
30675    1610612747
30676    1610612747
30677    1610612747
30678    1610612747
30679    1610612747
30680    1610612747
30681    1610612747
30682    1610612747
30683    1610612747
30684    1610612747
30685    1610612747


In [87]:
df1 = df[pd.notnull(df['is_goal'])]

In [88]:
df2 = df[pd.isnull(df['is_goal'])]

In [89]:
train_dataset = df1.sample(frac=0.99,random_state=0)
test_dataset = df1.drop(train_dataset.index)

In [90]:
train_labels = train_dataset.pop('is_goal')
test_labels = test_dataset.pop('is_goal')

In [91]:
train_stats = train_dataset.describe()
train_stats=train_stats.transpose()

In [92]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [93]:
# neural network model
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(normed_train_data.keys())]),
    layers.Dropout(0.2),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(0.2),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(0.2),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(0.2),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(0.2),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(0.2),
    layers.Dense(1, activation=tf.nn.sigmoid)
  ])

    optimizer = tf.keras.optimizers.Adam(0.0001)

    model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [94]:
model = build_model()

In [95]:
# Display training progress by printing a single dot for each completed epoch
tic=time.time()
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

EPOCHS = 500

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
    callbacks=[PrintDot()])
toc=time.time()


..........

In [96]:
# training accuracy
loss, mae, mse = model.evaluate(normed_train_data,train_labels, verbose=0)

print("Testing set Mean abs Error: {:5.2f}".format(mae))
print("Time taken for training:"+str(toc-tic))

Testing set Mean abs Error:  0.48
Time taken for training:16.240832090377808


In [97]:
# test accuracy
loss, mae, mse = model.evaluate(normed_test_data,test_labels, verbose=0)

print("Testing set Mean abs Error: {:5.2f}".format(mae))
print("Time taken for training:"+str(toc-tic))

Testing set Mean abs Error:  0.47
Time taken for training:16.240832090377808


In [98]:
z=df2.pop('is_goal')

In [99]:
# compiling result to csv
x=model.predict(norm(df2))
y=[]
for i in x:
    y.append(i[0])

In [100]:
dff=df2['shot_id_number'].copy()
dff=dff.to_frame().reset_index()

In [101]:
s = pd.Series(y)
df1 = pd.DataFrame({'is_goal':(s)})
dff=dff.join(df1)
# # dff.to_csv('submission_1.csv')
# dff.keys()

In [102]:
dff.pop('index')

0           0
1           7
2          16
3          19
4          21
5          32
6          33
7          34
8          35
9          36
10         37
11         44
12         49
13         54
14         59
15         61
16         65
17         66
18         70
19         71
20         75
21         79
22         84
23         85
24         86
25         91
26         94
27         96
28        103
29        112
        ...  
6238    30567
6239    30569
6240    30580
6241    30583
6242    30590
6243    30593
6244    30613
6245    30616
6246    30617
6247    30625
6248    30629
6249    30630
6250    30631
6251    30633
6252    30635
6253    30636
6254    30638
6255    30646
6256    30648
6257    30655
6258    30659
6259    30664
6260    30668
6261    30679
6262    30680
6263    30681
6264    30682
6265    30686
6266    30687
6267    30693
Name: index, Length: 6268, dtype: int64

In [103]:
dff.to_csv('submission.csv')